In [10]:
surname = "Новиков"  # Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1, шаг 5 - вариант: ", variant % 5 + 1)
print("Задача № 1, шаг 11 - вариант: ", variant % 2 + 1 )
print("задача № 2 - вариант: ", variant % 4 + 1)

Задача № 1, шаг 5 - вариант:  1
Задача № 1, шаг 11 - вариант:  2
задача № 2 - вариант:  4


In [ ]:
import pandas as pd

Согласно заданию необходимо:
    
1. Загрузить данные в `DataFrame`
2. Отсоровать данные по убыванию `pctrank`
3. Отобразите данные по индексу *WGI* за 2023 год в виде горизонтального столбчатого графика (`pctrank`).

In [11]:
# Загружаем данные
df1 = pd.read_excel("/home/ubuntu/homework/task_1/data/wgidataset.xlsx")
# Сортируем по столбцу `pctrank` (по возрастанию)
df.sort_values('pctrank', ascending=1)


,codeindyr,code,countryname,year,indicator,estimate,stddev,nsource,pctrank,pctranklower,pctrankupper
0,AFGcc1996,AFG,Afghanistan,1996,cc,-1.291705,0.340507,2,4.301075,0,27.419355
1,ALBcc1996,ALB,Albania,1996,cc,-0.893903,0.315914,3,19.354839,2.688172,43.010754
2,DZAcc1996,DZA,Algeria,1996,cc,-0.566741,0.262077,4,33.333332,16.666666,52.688171
3,ASMcc1996,ASM,American Samoa,1996,cc,..,..,..,..,..,..
4,ADOcc1996,ADO,Andorra,1996,cc,1.318143,0.480889,1,87.096771,72.043015,96.774193
...,...,...,...,...,...,...,...,...,...,...,...
32095,VIRva2023,VIR,Virgin Islands (U.S.),2023,va,..,..,..,..,..,..
32096,WBGva2023,WBG,West Bank and Gaza,2023,va,-1.118067,0.149837,6,18.137255,11.764706,24.509804
32097,YEMva2023,YEM,"Yemen, Rep.",2023,va,-1.550217,0.131432,8,6.372549,2.45098,11.764706
32098,ZMBva2023,ZMB,Zambia,2023,va,-0.047946,0.118482,12,45.098038,39.215687,52.450981
